In [1]:
import warnings
warnings.filterwarnings("ignore")
from copy import deepcopy
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import sys

sys.path.append("/code")

from tqdm import tqdm
import torch
# device = torch.device('cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# import gym
# import recogym

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

from sklearn.utils import check_random_state

# implementing OPE of the IPWLearner using synthetic bandit data
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

from scipy.special import softmax
import optuna
# from memory_profiler import profile


from estimators import (
    DirectMethod as DM
)

from simulation_utils import (
    eval_policy,
    generate_dataset,
    create_simulation_data_from_pi,
    get_train_data,
    get_opl_results_dict,
    CustomCFDataset,
    calc_reward
)

from models import (    
    CFModel,
    NeighborhoodModel,
    BPRModel, 
    RegressionModel
)

from training_utils import (
    fit_bpr,
    train,
    validation_loop
 )

from custom_losses import (
    SNDRPolicyLoss,
    BPRLoss
    )

random_state=12345
random_ = check_random_state(random_state)

Using device: cpu
Using device: cpu
Using device: cpu


In [17]:
pd.options.display.float_format = '{:,.8f}'.format

## `trainer_trial` Function

This function runs policy learning experiments using offline bandit data and evaluates various estimators.

### Parameters
- **num_runs** (int): Number of experimental runs per training size
- **num_neighbors** (int): Number of neighbors to consider in the neighborhood model
- **num_rounds_list** (list): List of training set sizes to evaluate
- **dataset** (dict): Contains dataset information including embeddings, action probabilities, and reward probabilities
- **batch_size** (int): Batch size for training the policy model
- **num_epochs** (int): Number of training epochs for each experiment
- **lr** (float, default=0.001): Learning rate for the optimizer

### Process Flow
1. Initializes result structures and retrieval models
2. For each training size in `num_rounds_list`:
   - Creates a uniform logging policy and simulates data
   - Generates training data for offline learning
   - Fits regression and neighborhood models for reward estimation
   - Initializes and trains a counterfactual policy model
   - Evaluates policy performance using various estimators
   - Collects metrics on policy reward and embedding quality

### Returns
- **DataFrame**: Results table with rows indexed by training size and columns for various metrics:
  - `policy_rewards`: True expected reward of the learned policy
  - Various estimator errors (`ipw`, `reg_dm`, `conv_dm`, `conv_dr`, `conv_sndr`)
  - Variance metrics for each estimator
  - Embedding quality metrics comparing learned representations to ground truth

### Implementation Notes
- Uses uniform random logging policy for collecting offline data
- Employs Self-Normalized Doubly Robust (SNDR) policy learning
- Measures embedding quality via RMSE to original/ground truth embeddings

In [3]:
def trainer_trial(
                  num_runs,
                  num_neighbors,
                  num_rounds_list,
                  dataset,
                  batch_size,
                  val_size=2000
                  ):
    
    # Define device at the beginning
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    def objective(trial):
    
        # Optuna objective function
        lr = trial.suggest_loguniform("lr", 1e-4, 1e-3)
        epochs = trial.suggest_int("num_epochs", 1, 10)

        trial_neigh_model = NeighborhoodModel(
                                                train_data['x_idx'],
                                                train_data['a'], 
                                                our_a,
                                                our_x, 
                                                train_data['r'], 
                                                num_neighbors=num_neighbors
                                            )
        

        trial_model = CFModel(
                        n_users, 
                        n_actions, 
                        emb_dim, 
                        initial_user_embeddings=torch.tensor(our_x, device=device), 
                        initial_actions_embeddings=torch.tensor(our_a, device=device)
                        )
        
        # Training
        train(trial_model, train_loader, trial_neigh_model, criterion=SNDRPolicyLoss(), num_epochs=epochs, lr=lr, device=device)
        return validation_loop(trial_model, val_loader, trial_neigh_model)
    
    device = torch.device('cpu')

    dm = DM()
    results = {}

    our_x, our_a = dataset["our_x"], dataset["our_a"]
    emb_x, emb_a = dataset["emb_x"], dataset["emb_a"]
    original_x, original_a = dataset["original_x"], dataset["original_a"]
    n_users, n_actions, emb_dim = dataset["n_users"], dataset["n_actions"], dataset["emb_dim"]
    
    first = True

    for train_size in num_rounds_list:
        reg_results, conv_results = [], []
        
        for run in range(num_runs):

            pi_0 = softmax(our_x @ our_a.T, axis=1)
            original_policy_prob = np.expand_dims(pi_0, -1)

            simulation_data = create_simulation_data_from_pi(
                                                            dataset,
                                                            pi_0,
                                                            train_size + val_size,
                                                            random_state=(run + 1) * train_size
                                                            )
            # test_data = get_test_data(dataset, simulation_data, n_test_data)

            # idx = np.arange(train_size) + n_test_data
            idx = np.arange(train_size)

            train_data = get_train_data(n_actions, train_size, simulation_data, idx, our_x)
            val_data = get_train_data(n_actions, val_size, simulation_data, np.arange(val_size) + train_size, our_x)

            regression_model = RegressionModel(
                                    n_actions=n_actions,
                                    action_context=our_x,
                                    base_model=LogisticRegression(random_state=12345)
                                    )
            
            regression_model.fit(
                                train_data['x'], 
                                train_data['a'],
                                train_data['r'],
                                original_policy_prob[train_data['x_idx'],
                                train_data['a']].squeeze()
                                )
            
            neighberhoodmodel = NeighborhoodModel(
                                                    train_data['x_idx'],
                                                    train_data['a'], 
                                                    our_a,
                                                    our_x, 
                                                    train_data['r'], 
                                                    num_neighbors=num_neighbors
                                                )
            

            model = CFModel(
                            n_users, 
                            n_actions, 
                            emb_dim, 
                            initial_user_embeddings=torch.tensor(our_x, device=device), 
                            initial_actions_embeddings=torch.tensor(our_a, device=device)
                            )
            
            cf_dataset =  CustomCFDataset(
                                       train_data['x_idx'], 
                                       train_data['a'], 
                                       train_data['r'], 
                                       original_policy_prob
                                       )
            
            train_loader = DataLoader(cf_dataset, batch_size=batch_size, shuffle=True)

            val_dataset =  CustomCFDataset(
                            val_data['x_idx'], 
                            val_data['a'], 
                            val_data['r'], 
                            original_policy_prob
                            )
            
            val_loader = DataLoader(val_dataset, batch_size=val_size, shuffle=True)

            if first:
                policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)
                conv_results.append(eval_policy(neighberhoodmodel, train_data, original_policy_prob, policy))
                conv_results[-1] = np.append(calc_reward(dataset, policy), conv_results[-1])
                conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_a-our_a)**2)), np.sqrt(np.mean((original_a-our_a)**2))])
                conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_x-our_x)**2)), np.sqrt(np.mean((original_x-our_x)**2))])
                # bpr_scores = bpr_model.calc_scores(torch.tensor(train_data['x_idx'], device=device, dtype=torch.long)).detach().cpu().numpy()
                reg_dm = dm.estimate_policy_value(policy[train_data['x_idx']], regression_model.predict(train_data['x']))
                reg_results.append(reg_dm)
                first = False
                reg_results = np.array(reg_results)
                conv_results = np.array(conv_results)
                results[0] = get_opl_results_dict(reg_results, conv_results)
                reg_results, conv_results = [], []
                
            # Bloss = BPRLoss()
            # bpr_model = BPRModel(
            #                     n_users,
            #                     n_actions,
            #                     emb_x.shape[1], 
            #                     initial_user_embeddings=torch.tensor(our_x, device=device), 
            #                     initial_actions_embeddings=torch.tensor(our_a, device=device)
            #                     )

            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=10, show_progress_bar=True)
            best_params = study.best_params

            neighberhoodmodel = NeighborhoodModel(
                                                    train_data['x_idx'],
                                                    train_data['a'], 
                                                    our_a,
                                                    our_x, 
                                                    train_data['r'], 
                                                    num_neighbors=num_neighbors
                                                )
            

            model = CFModel(
                            n_users, 
                            n_actions, 
                            emb_dim, 
                            initial_user_embeddings=torch.tensor(our_x, device=device), 
                            initial_actions_embeddings=torch.tensor(our_a, device=device)
                            )

            train(model, train_loader, neighberhoodmodel, criterion=SNDRPolicyLoss(), num_epochs=best_params['num_epochs'], lr=best_params['lr'], device=device)
            # fit_bpr(bpr_model, Bloss, train_loader, num_epochs=3, lr=0.001, device=device)
            # neighborhood_model.update(model.get_params()[0].detach().numpy(), model.get_params()[1].detach().numpy())'

            our_x, our_a = model.get_params()
            our_a, our_x = our_a.detach().cpu().numpy(), our_x.detach().cpu().numpy()

            policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)

            # bpr_scores = bpr_model.calc_scores(torch.tensor(train_data['x_idx'], device=device, dtype=torch.long)).detach().cpu().numpy()
            # reg_dm = dm.estimate_policy_value(policy[train_data['x_idx']], bpr_scores)
            reg_dm = dm.estimate_policy_value(policy[train_data['x_idx']], regression_model.predict(train_data['x']))

            reg_results.append(reg_dm)

            # conv_results.append(eval_policy(neighberhoodmodel, test_data, original_policy_prob[test_data['x_idx']], policy))
            conv_results.append(eval_policy(neighberhoodmodel, train_data, original_policy_prob, policy))

            conv_results[-1] = np.append(calc_reward(dataset, policy), conv_results[-1])
            conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_a-our_a)**2)), np.sqrt(np.mean((original_a-our_a)**2))])

            # temp.append(np.mean((emb_a-our_a)**2, axis=0))

            conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_x-our_x)**2)), np.sqrt(np.mean((original_x-our_x)**2))])
            
            our_a, our_x = original_a.copy(), original_x.copy()

        reg_results = np.array(reg_results)
        conv_results = np.array(conv_results)

        results[train_size] = get_opl_results_dict(reg_results, conv_results)
    
    return pd.DataFrame.from_dict(results, orient='index')

## Learning

We will run several simulations on a generated dataset, the dataset is generated like this:
$$ \text{We have users U and actions A } u_i \sim N(0, I_{emb_dim}) \ a_i \sim N(0, I_{emb_dim})$$
$$ p_{ij} = 1 / (5 + e^{-(u_i.T a_j)}) $$
$$r_{ij} \sim Bin(p_{ij})$$

We have a policy $\pi$
and it's ground truth reward is calculated by
$$R_{gt} = \sum_{i}{\sum_{j}{\pi_{ij} * p_{ij}}} $$

Our parameters for the dataset will be
$$EmbDim = 5$$
$$NumActions= 150$$
$$NumUsers = 150$$
$$NeighborhoodSize = 6$$

to learn a new policy from $\pi$ we will sample from:
$$\pi_{start} = (1-\epsilon)*\pi + \epsilon * \pi_{random}$$

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [5]:
num_runs = 1

In [6]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 4,
                    # sigma = 0.1,
                    eps = 0.2, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.02
                    )

train_dataset = generate_dataset(dataset_params)

CTR: 0.01861246403332251


In [7]:
train_dataset.keys()

dict_keys(['emb_a', 'our_a', 'original_a', 'emb_x', 'our_x', 'original_x', 'q_x_a', 'n_actions', 'n_users', 'emb_dim', 'user_prior'])

In [8]:
num_runs = 1
batch_size = 200
num_neighbors = 51
num_rounds_list = [30000, 60000, 80000, 90000]

### 1

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.005$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [9]:
df4 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=35000)

[I 2025-08-18 18:19:04,422] A new study created in memory with name: no-name-37065847-3816-4ca5-88de-f8fbac65bbee
Best trial: 0. Best value: 0.0104472:  10%|█         | 1/10 [01:26<13:02, 86.91s/it]

[I 2025-08-18 18:20:31,332] Trial 0 finished with value: 0.010447228179581563 and parameters: {'lr': 0.0007220985987695284, 'num_epochs': 3}. Best is trial 0 with value: 0.010447228179581563.


Best trial: 1. Best value: 0.0105225:  20%|██        | 2/10 [02:54<11:38, 87.31s/it]

[I 2025-08-18 18:21:58,925] Trial 1 finished with value: 0.01052248297019858 and parameters: {'lr': 0.0002005232834880863, 'num_epochs': 3}. Best is trial 1 with value: 0.01052248297019858.


Best trial: 1. Best value: 0.0105225:  30%|███       | 3/10 [04:19<10:02, 86.03s/it]

...

[I 2025-08-18 20:36:47,068] Trial 7 finished with value: 0.01297829090891596 and parameters: {'lr': 0.00018171657505336683, 'num_epochs': 1}. Best is trial 6 with value: 0.012982906051217446.


Best trial: 6. Best value: 0.0129829:  90%|█████████ | 9/10 [39:57<04:26, 266.26s/it]

[I 2025-08-18 20:41:16,631] Trial 8 finished with value: 0.012420259294756687 and parameters: {'lr': 0.0001831293610935004, 'num_epochs': 8}. Best is trial 6 with value: 0.012982906051217446.


Best trial: 6. Best value: 0.0129829: 100%|██████████| 10/10 [44:22<00:00, 266.22s/it]


[I 2025-08-18 20:45:41,026] Trial 9 finished with value: 0.011867784048141252 and parameters: {'lr': 0.00024321546575788123, 'num_epochs': 8}. Best is trial 6 with value: 0.012982906051217446.


In [18]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.01978745,0.01935893,0.01978458,0.01945779,0.01965838,0.02030646,0.27840712,0.00000000,0.23742245,0.00000000
30000,0.01978824,0.01933884,0.01978477,0.01945753,0.01965032,0.02027674,0.27723678,0.00853207,0.23751290,0.00710262
60000,0.01979021,0.02009584,0.01951256,0.01944882,0.01957111,0.01997048,0.27598104,0.04029597,0.24018790,0.03518278
80000,0.01978966,0.01938873,0.01942306,0.01962110,0.01951601,0.01917503,0.27621442,0.02002988,0.23755858,0.01781210
90000,0.01978770,0.01888049,0.01933169,0.01926089,0.01918405,0.01893465,0.27808682,0.00276144,0.23740673,0.00245908


In [19]:
df4

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,ipw_var,reg_dm_var,conv_dm_var,conv_dr_var,conv_sndr_var,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.01978745,0.01935893,0.01978458,0.01945779,0.01965838,0.02030646,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.27840712,0.00000000,0.23742245,0.00000000
30000,0.01978824,0.01933884,0.01978477,0.01945753,0.01965032,0.02027674,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.27723678,0.00853207,0.23751290,0.00710262
60000,0.01979021,0.02009584,0.01951256,0.01944882,0.01957111,0.01997048,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.27598104,0.04029597,0.24018790,0.03518278
80000,0.01978966,0.01938873,0.01942306,0.01962110,0.01951601,0.01917503,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.27621442,0.02002988,0.23755858,0.01781210
90000,0.01978770,0.01888049,0.01933169,0.01926089,0.01918405,0.01893465,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.27808682,0.00276144,0.23740673,0.00245908


In [12]:
num_rounds_list = [3000, 6000, 8000, 9000]

### 2

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.001$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [13]:
df5 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=3500)

[I 2025-08-18 20:50:01,245] A new study created in memory with name: no-name-5bf582bb-8c04-4208-9d37-1fad2006ccda
Best trial: 0. Best value: 0.00553381:  10%|█         | 1/10 [00:07<01:08,  7.63s/it]

[I 2025-08-18 20:50:08,873] Trial 0 finished with value: 0.00553381045182111 and parameters: {'lr': 0.0002062836169406708, 'num_epochs': 9}. Best is trial 0 with value: 0.00553381045182111.


Best trial: 1. Best value: 0.00554369:  20%|██        | 2/10 [00:15<01:01,  7.64s/it]

[I 2025-08-18 20:50:16,517] Trial 1 finished with value: 0.0055436879852006245 and parameters: {'lr': 0.0006463335099062945, 'num_epochs': 7}. Best is trial 1 with value: 0.0055436879852006245.


Best trial: 1. Best value: 0.00554369:  30%|███       | 3/10 [00:23<00:53,  7.71s/it]

...

[I 2025-08-18 21:02:37,657] Trial 7 finished with value: 0.009358228524662995 and parameters: {'lr': 0.00011333756556964095, 'num_epochs': 6}. Best is trial 2 with value: 0.00937063337656717.


Best trial: 8. Best value: 0.00939558:  90%|█████████ | 9/10 [03:37<00:24, 24.18s/it]

[I 2025-08-18 21:03:02,019] Trial 8 finished with value: 0.009395580579119099 and parameters: {'lr': 0.000402505762171222, 'num_epochs': 5}. Best is trial 8 with value: 0.009395580579119099.


Best trial: 8. Best value: 0.00939558: 100%|██████████| 10/10 [04:01<00:00, 24.16s/it]


[I 2025-08-18 21:03:25,939] Trial 9 finished with value: 0.009373266705966847 and parameters: {'lr': 0.0006123613550065155, 'num_epochs': 1}. Best is trial 8 with value: 0.009395580579119099.


In [20]:
df5

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,ipw_var,reg_dm_var,conv_dm_var,conv_dr_var,conv_sndr_var,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.01978745,0.01964856,0.01969683,0.01992051,0.01994549,0.02002184,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.27840712,0.00000000,0.23742245,0.00000000
3000,0.01978779,0.01965154,0.01969771,0.01992277,0.01994686,0.02002081,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.27785089,0.00248316,0.23739990,0.00153581
6000,0.01978885,0.02199032,0.02274482,0.02273408,0.02284151,0.02317750,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.27619045,0.01298967,0.23773052,0.00882750
8000,0.01979069,0.02177713,0.02005721,0.02016991,0.02057855,0.02193945,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.27391127,0.02429254,0.23816576,0.01751989
9000,0.01978977,0.02362445,0.02127779,0.02114514,0.02142483,0.02234303,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.27558966,0.01660720,0.23741756,0.01266007


### 3

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.003$$
$$n_{epochs} = 10$$
$$BatchSize=50$$

In [15]:
df6 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size)

[I 2025-08-18 21:03:57,328] A new study created in memory with name: no-name-bfa9bf7d-151c-4705-87f9-e0dc42e00dbf
Best trial: 0. Best value: 0.0125066:  10%|█         | 1/10 [00:07<01:05,  7.24s/it]

[I 2025-08-18 21:04:04,570] Trial 0 finished with value: 0.012506563033600631 and parameters: {'lr': 0.0007602033817158572, 'num_epochs': 6}. Best is trial 0 with value: 0.012506563033600631.


Best trial: 1. Best value: 0.0125324:  20%|██        | 2/10 [00:14<00:58,  7.25s/it]

[I 2025-08-18 21:04:11,832] Trial 1 finished with value: 0.01253241233339062 and parameters: {'lr': 0.00038428445979786947, 'num_epochs': 4}. Best is trial 1 with value: 0.01253241233339062.


Best trial: 1. Best value: 0.0125324:  30%|███       | 3/10 [00:21<00:51,  7.36s/it]

...

Best trial: 3. Best value: 0.0100327:  50%|█████     | 5/10 [02:06<02:07, 25.49s/it]

[I 2025-08-18 21:15:13,100] Trial 4 finished with value: 0.010018433625815414 and parameters: {'lr': 0.00017768238470758355, 'num_epochs': 3}. Best is trial 3 with value: 0.010032749896563666.


Best trial: 3. Best value: 0.0100327:  50%|█████     | 5/10 [02:19<02:19, 27.90s/it]


[W 2025-08-18 21:15:26,335] Trial 5 failed with parameters: {'lr': 0.0006929989248602841, 'num_epochs': 2} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/roee/Documents/code/OPC/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_2093602/3301235813.py", line 18, in objective
    trial_neigh_model = NeighborhoodModel(
                        ^^^^^^^^^^^^^^^^^^
  File "/home/roee/Documents/code/OPC/models.py", line 37, in __init__
    self.fit(action_emb, context_emb, actions, context, rewards)
  File "/home/roee/Documents/code/OPC/models.py", line 44, in fit
    self.calculate_scores()
  File "/home/roee/Documents/code/OPC/models.py", line 63, in calculate_scores
    self.scores = self.context_convolve(context)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/roee/Documents/code/OPC/models.py", 

KeyboardInterrupt: 

In [ ]:
df6

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,ipw_var,reg_dm_var,conv_dm_var,conv_dr_var,conv_sndr_var,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.0140,0.0206,0.0158,0.0163,0.0170,0.0190,0.0000,0.0000,0.0000,0.0000,0.0000,0.2189,0.0000,0.3154,0.0000
3000,0.0139,0.0206,0.0158,0.0163,0.0170,0.0190,0.0000,0.0000,0.0000,0.0000,0.0000,0.2189,0.0000,0.3154,0.0000
6000,0.0140,0.0156,0.0129,0.0135,0.0135,0.0136,0.0000,0.0000,0.0000,0.0000,0.0000,0.2079,0.0600,0.3195,0.0415
8000,0.0138,0.0194,0.0155,0.0168,0.0167,0.0163,0.0000,0.0000,0.0000,0.0000,0.0000,0.2189,0.0001,0.3154,0.0001
9000,0.0143,0.0164,0.0150,0.0165,0.0160,0.0146,0.0000,0.0000,0.0000,0.0000,0.0000,0.2155,0.0121,0.3156,0.0089


### 4

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.05$$
$$n_{epochs} = 10$$
$$BatchSize=150$$

In [ ]:
df7 = trainer_trial(num_runs, num_neighbors, num_rounds_list[:-3], train_dataset, batch_size+100, num_epochs=10, lr=0.05)

TypeError: trainer_trial() got an unexpected keyword argument 'num_epochs'

In [ ]:
df7